<h1>¿Influyen la cantidad de reviews en la calificación de un libro?</h1>

Sin duda alguna, el crecimiento exponencial del conocimiento humano fue causado por la capacidad de transmitir ideas a través del tiempo. Gracias a la palabra escrita, la humanidad no solo ha sido capaz de transmitir conocimiento, sino creó una herramienta que permití a las personas vivir experiencias ajenas de formas inimaginables para quién no ha leído una buena novela o cuento. No se requiere ser bibliómano para poder recomendar un libro, alegando que es "una joya" de la literatura. Todos nos hemos vueltos profetas y evangelizadores de algún buen libro. Pero si el gusto es relativo, ¿podrá ser que a alguien no le guste el mismo libro que a mí me inspiro a ser mejor persona? Claro que sí, es muy probable que al menos a una persona no le guste. Sin embargo, no quiere decir que el hecho de que no lo considere una "joya de la literatura", la otra persona lo odié, como todo en la vida, es bueno tener matices.

La escala de 5 estrellas es conocida por todo el mundo, dónde se entiende en acuerdo social que 0 es un rotundo "no lo recomiendo" cerca del "si quieres un consejo, no lo leas"; 3 se puede entender como un "bueno, pero no el mejor"; mientras que 5, la calificicación de la excelencia. Brindándole a la gente estos matices, podemos obtener más información, así como nuevas preguntas, dónde la más importante podría ser: ¿Mientras más gente lea un libro, más difícil será alcanzar las 5 estrellas?

Con el Dataset de esta página, se plantea hacer un estudio estadístico sobre la relación que tiene el promedio de calificación que posee un libro, usando la escala de rating de 5 estrellas ('average_rating') y el número de reseñas que ha recibido el mismo ('text_reviews_coun'), considerando el número de reseñas como personas que han leído el libro.

<h1>Read Data</h1>

Como primer paso para acceder a la información Desde un punto inicial se poceden a importar las librerías del set básico para analizar un DataFrame con Pandas así como las bibliotecas para el uso de gráficas y arreglos.

In [ ]:
import scipy.stats
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
books = pd.read_csv('../input/bookscsv/books.csv')

In [ ]:
books.head()

In [ ]:
books.columns

Al observar el encabezado de la tabla, se puede apreciar que existe una columna de "sobra". El hecho de que lleve de nombre "Unnamed: 12" indica que esta columna es el resultado de que una o más filas contengan una coma de más. Considerando la naturaleza de los datos, se puede deducir que el error es más probable de aparecer en el título del libro, o en los autores. Dadas estas suposiciones, se buscará analizar las filas que muestren estos errores, para poder identificar corroborar que si sólo es en una columna o en dos.

Para ello, se diseña una función que busque estos "errores"; si los encuentra, imprime la fila, pero en caso de que tenga un error de índice por no contener información, los pasará de largo y seguirá.

In [ ]:
def data_verifier(file):
    with open(books, mode="r", encoding='utf-8', newline='') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        header = next(csv_reader)
        count = 0
        for row in csv_reader:
            try:
                last_row = row[12]
                if last_row == '':
                    last_row = row[13]
                print(f'{row}')
                print(f'LENs. title: {len(row[1])} authors: {len(row[2])} sup_extra: {len(row[3])} sup_rating: {len(row[4])}\n')
             
            except IndexError:

                pass
        
        csv_file.close()

Con la función diseñada (y probada en script), se procede a la importación de csv y os para que el Kernel no lo señale como un objeto tipo DataFrame y lo maneje con un file y string.

In [ ]:
import csv, os

books = '../input/goodreadsbooks/books.csv'
data_verifier(books)

Gracias a esta función, podemos determinar que el error se encuentra en la columna de autores. Si bien, las longitudes sobresalen en el segundo error, al leer la información detalladamente podemos observar que el error sucede por una coma extra en autores. Ya que se realizará un estudio sobre la relación entre el rating y las reviews, podemos precindir de toda información que no sea estos dos valores y el título.

<h1>Carga de la información y análisis de la información</h1>

<h2>Delimitación de la información necesaria</h2>

Haciendo un análisis superficial, solo se requerirá la información del rating (float), reviews (int) y los libros resultantes de estos análisis. En el caso del rating y las reviews, podemos notar que un parámetro siempre solicitado es el valor máximo. Por lo que al momento de cargar la información, podemos aprovechar para obtenerlo dentro del mismo ciclo. Si el lector desea agregar más información, a continuación se deja el chunk de la función load_and_max para su edición.

Se hace la aclaración que la importación de 'csv' y 'os' es solo de apoyo didáctico, ya que basta con la importación que se hizo previamente.

In [ ]:
import csv, os

def load_and_max(file, column_to_find):
    with open(books, mode="r", encoding='utf-8', newline='') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        header = next(csv_reader)
        column_array = []
        max_value = 0

        for row in csv_reader:
            try:
                last_row = row[12]
                if last_row == '':
                    last_row = row[13]

                if column_to_find == 'average_rating': column_data = float(row[4])
                elif column_to_find == 'num_pages': column_data = int(row[8])
                elif column_to_find == 'ratings_count': column_data = int(row[9])
                elif column_to_find == 'text_reviews_count': column_data = int(row[10])
                else:
                    column_array = 'Data not Found. Check input'
                    max_value = 0
                    return column_array, max_value
            except IndexError:
                if column_to_find == 'average_rating': column_data = float(row[3])
                elif column_to_find == 'num_pages': column_data =  int(row[7])
                elif column_to_find == 'ratings_count': column_data = int(row[8])
                elif column_to_find == 'text_reviews_count': column_data = int(row[9])
                else:
                    column_array = 'Data not Found. Check input'
                    max_value = 0
                    return column_array, max_value

            column_array.append(column_data)
            if column_data > max_value:
                max_value = column_data

    csv_file.close()

    return column_array, max_value

Cómo es posible de observar, la función previa solo fue diseñada para reducir tiempos de ejecución, ya que de esta forma se ahorran dos ciclos O(n) cada uno para la obtención de los valores máximos de los arrays. en caso de que se requiera algún otro dato de la columna, se podría usar la función que se encuentra a continuación, considerando que:

1. Solo carga un array a la vez.
1. No obtiene valores máximos.
1. Se garantiza su desempeño exclusivamente para este caso con este csv. Para otro tipo de csv's su utilidad y eficiencia no está garantizada.

In [ ]:
def load_array(file, column_to_find):

    with open(books, mode="r", encoding='utf-8', newline='') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)
        column_array = []

        for row in csv_reader:
            try:
                last_row = row[12]
                if last_row == '':
                    last_row = row[13]
                #Ordered followinf the fild
                #First Strings
                if column_to_find == 'title': column_data = row[1]
                elif column_to_find == 'authors': column_data = row[2] + ',' + row[3]
                #Float
                elif column_to_find == 'average_rating': column_data = float(row[4])
                #Strings hided as Int
                elif column_to_find == 'isbn': column_data = row[5]
                elif column_to_find == 'isbn13': column_data = row[6]
                #String AF
                elif column_to_find == 'language_code': column_data = row[7]
                elif column_to_find == '  num_pages': column_data = int(row[8])
                elif column_to_find == 'ratings_count': column_data = int(row[9])
                elif column_to_find == 'text_reviews_count': column_data = int(row[10])
                #Reason you import Datetime
                elif column_to_find == 'publication_date': column_data = datetime.strptime(row[11], '%m/%d/%Y')
                #String
                elif column_to_find == 'publisher': column_data = row[12]
                else:
                    column_data = 'Data not Found. Check input'
                    
                    return column_array

                    
            except IndexError:
                #Ordered followinf the fild
                #First Strings
                if column_to_find == 'title': column_data = row[1]
                elif column_to_find == 'authors': column_data = row[2]
                #Float
                elif column_to_find == 'average_rating': column_data = float(row[3])
                #Strings hided as Int
                elif column_to_find == 'isbn': column_data = row[4]
                elif column_to_find == 'isbn13': column_data = row[5]
                #String AF
                elif column_to_find == 'language_code': column_data = row[6]
                elif column_to_find == 'num_pages': column_data =  int(row[7])
                elif column_to_find == 'ratings_count': column_data = int(row[8])
                elif column_to_find == 'text_reviews_count': column_data = int(row[9])
                #Reason you import Datetime
                elif column_to_find == 'publication_date': column_data = datetime.strptime(row[10], '%m/%d/%Y')
                #String
                elif column_to_find == 'publisher': column_data = row[11]
                else:
                    column_data = 'Data not Found. Check input'
                    
                    return column_array

            column_array.append(column_data)
        

    csv_file.close()

    return column_array

Ejecutando por partes el código, se procede a analizarlo su ejecución como partes de un script simple de python, obteniendo la ubicación de nuestro archivo y extrayendo la información relevante.

In [ ]:
books = '../input/goodreadsbooks/books.csv'

titles = load_array(books, 'title')
rating, max_rating = load_and_max(books, 'average_rating')
reviews, max_reviews = load_and_max(books, 'text_reviews_count')

Para tener noción de que no se omitió ningún dato, se solicita su tamaño ya que, si tuvieran una diferencia por un solo dato, se corre el riesgo de obtener un error de índice en futuros análisis. De igual forma, se garantiza que los datos faltantes que puedan existir, son por error en el archivo y no por la carga, reduciendo la posibilidad de un re-trabajo

In [ ]:
print(f'Información disponible: Titulos: {len(titles)}, rating: {len(rating)}, reviews: {len(reviews)}')

Al tener el mismo tamaño, procedemos a hacer el análisis con ayuda de Pandas.

In [ ]:
books_dict = {'rating': rating,
             'reviews': reviews,
             'titles': titles}

books_df = pd.DataFrame(books_dict, columns=['rating','reviews','titles'])
books_df

<h3>Análisis del Rating</h3>

Antes de realizar conjenturas sobre su calentamiento, encontrando un soporte visual en un histograma, se puede apreciar de forma rápida la tendencia que tiene la serie del rating, permitiéndo decidir cuál es la vía más eficiente para el análisis. Asignando 10 bins, se entiende que el gráfico x muestra la distribución del rating en saltos de 0.5 en la calificación; 100, 0.05; y 500, 0.01 (todas las posibles calificaciones)

In [ ]:
y = books_df['rating']

fig, ax = plt.subplots()
ax.hist(y, bins = 500)
ax.set_xlabel('rating')
ax.set_ylabel('Frecuencia')

plt.axvline(np.mean(y)-np.std(y), c = 'k', linestyle = ':', label = '-1 desv. std.')
plt.axvline(np.mean(y), c = 'r', linestyle = '-', label = 'Promedio')
plt.axvline(np.mean(y)+np.std(y), c = 'k', linestyle = ':', label = '+1 desv. std.')
ax.legend()

In [ ]:
y = books_df['rating']

fig, ax = plt.subplots()
ax.hist(y, bins = 100)
ax.set_xlabel('rating')
ax.set_ylabel('Frecuencia')

plt.axvline(np.mean(y)-np.std(y), c = 'k', linestyle = ':', label = '-1 desv. std.')
plt.axvline(np.mean(y), c = 'r', linestyle = '-', label = 'Promedio')
plt.axvline(np.mean(y)+np.std(y), c = 'k', linestyle = ':', label = '+1 desv. std.')
ax.legend()

In [ ]:
y = books_df['rating']

fig, ax = plt.subplots()
ax.hist(y, bins = 10)
ax.set_xlabel('rating')
ax.set_ylabel('Frecuencia')

plt.axvline(np.mean(y)-np.std(y), c = 'k', linestyle = ':', label = '-1 desv. std.')
plt.axvline(np.mean(y), c = 'r', linestyle = '-', label = 'Promedio')
plt.axvline(np.mean(y)+np.std(y), c = 'k', linestyle = ':', label = '+1 desv. std.')
ax.legend()

Para el caso del rating se puede observar una clara tendencia a la derecha, indicando que resulta más fácil encontrar un valor cercano a las 4 estrellas, pero que, al obtenerlo, decrece de forma más drástica. El hecho de encontrar pocos casos de calificaciones menores a 3.5 podría indicar que la gente tiende a reseñar los libros que les ha parecido decentes, pero resulta raro que alguién realice una reseña de algún libro que no le gustó. A simple vista, se aprecia que es muy raro encontrar una calificación mayor a 4.5 estrellas, pero que sí existen libros con 5 estrellas ("excelente").

Con esa información, se procede a analizar cuales son los libros con mayor calificación (5 estrellas)

In [ ]:
def match_case_condition(array, value):
    match_one_condition = []
    for element_ub in range(len(array)):
        if array[element_ub] == value:
            match_one_condition.append(element_ub)

    return match_one_condition

In [ ]:
maxium_books = match_case_condition(rating,5)

verified_rate = []
for ub in maxium_books:
    verified_rate.append(rating[ub])

n_reviews_of_titles = []
for ub in maxium_books:
    n_reviews_of_titles.append(reviews[ub])    
    
best_rated_titles = []
for ub in maxium_books:
    best_rated_titles.append(titles[ub])
    
best_rated_books_dict = {'rating': verified_rate,
             'reviews': n_reviews_of_titles,
             'titles': best_rated_titles}

best_rated_books_df = pd.DataFrame(best_rated_books_dict, columns=['rating','reviews','titles'])
best_rated_books_df

Con este análisis, podemos afirmar que es falsa la idea de que los libros mejor calificados son los que tienen el mayor número de lectores. Este análisis inclusive puede poner en mesa la idea de que un libro con 5 estrellas ha sido poco evaluado por sus lectores.

<h2>Considerando Rating y Reviews</h2>

A diferencia de las estrellas, los reviews tienen la característica de poder tener un comportamiento exponencial, por lo que clasificarlos por grupos carecería de sentido. Para ello se muestran las siguientes gráficas, dónde la primera muestra una clasificación en 10 categorías y la siguiente de 100, usando sus valores cómo parámetro clasificador.

In [ ]:
y = books_df['reviews']
fig, ax = plt.subplots()
ax.hist(y, bins = 10)
ax.set_xlabel('reviews')
ax.set_ylabel('Frecuencia')

plt.axvline(np.mean(y)-np.std(y), c = 'k', linestyle = ':', label = '-1 desv. std.')
plt.axvline(np.mean(y), c = 'r', linestyle = '-', label = 'Promedio')
plt.axvline(np.mean(y)+np.std(y), c = 'k', linestyle = ':', label = '+1 desv. std.')
ax.legend()

In [ ]:
y = books_df['reviews']
fig, ax = plt.subplots()
ax.hist(y, bins = 100)
ax.set_xlabel('reviews')
ax.set_ylabel('Frecuencia')

plt.axvline(np.mean(y)-np.std(y), c = 'k', linestyle = ':', label = '-1 desv. std.')
plt.axvline(np.mean(y), c = 'r', linestyle = '-', label = 'Promedio')
plt.axvline(np.mean(y)+np.std(y), c = 'k', linestyle = ':', label = '+1 desv. std.')
ax.legend()

Aparte de mostrar la casi inexistencia de valores altos, se puede deducir que un libro no popular tendrá escasas receñas; y el caso contrario con los libros populares, al poseer muchas receñas. Gracias a la noción otorgada por los resultados con el gráfico, podríamos obtener los libros que tengan como mínimo 1,000 reviews. Con esto, se reducen las iteraciones para encontrar un Top en el que los libros con mayor reviews tengan buena calificación. Esta función, de igual forma, podría ser útil para encontrar todos los libros con la calificación de nuestro interés.

In [ ]:
def merge_sort(array):
    if len(array) > 1:
        middle = len(array) // 2
        left = array[:middle]
        right = array[middle:]

        merge_sort(left)
        merge_sort(right)
        
        """SubArrays Iterators"""
        i = 0
        j = 0
        """MainArray Iterator"""
        k = 0

        while i < len(left) and j < len(right):
            if left[i] < right[j]:
                array[k] = left[i]
                i += 1
            else:
                array[k] = right[j]
                j += 1
            
            k += 1

        while i < len(left):
            array[k] = left[i]
            i += 1
            k += 1

        while j < len(right):
            array[k] = right[j]
            j += 1
            k += 1

    return array

def binary_search(array, start, end, search_value):
    if start > end:
        return end
    
    middle = (start + end) // 2

    if array[middle] == search_value:
        return middle
    elif array[middle] < search_value:
        return binary_search(array, middle + 1, end, search_value)
    else:
        return binary_search(array, start, middle - 1, search_value)

    
def top_condisioned(array, start_value):
    helper_array = sorted_set(array.copy())
    helpers_end = len(helper_array) - 1
    ubication =  binary_search(helper_array, 0, helpers_end, start_value)
    
    top_condisioned = []
    for i in range(ubication, helpers_end):
        top_condisioned.append(helper_array[i])

    return top_condisioned

def counted_array(elements_to_count, original_array):
    count_array = []
    for value in range(len(elements_to_count)):
        count_array.append(original_array.count(elements_to_count[value]))
    
    return count_array

def sorted_set(array):
    reduced_set = set(array)
    reduced_array = []
    for element in reduced_set:
        reduced_array.append(element)
    reduced_array = merge_sort(reduced_array)

    return reduced_array

In [ ]:
topc_reviews = top_condisioned(reviews, 1000)
print(f'Cantidad de libros con más de 1,000 reviews: {len(topc_reviews)}')

Debido al valor alto, comparado con el número de calificaciones de 5 estrellas, de más de 1,000 receñas, y recordando lo escasos que son los libros con calificaciones mayores a 4.5, podemos buscar los libros que hagan match con estas dos ideas. Primero obteniendo el arreglo con calificaciones mayores a 4.5

In [ ]:
topc_rating = top_condisioned(rating, 4.5)
print(f'Cantidad de calificaciones disponibles y obtenidas, que sean mayores o igual a 4.5 = {len(topc_rating)}; las cuales son: \n{topc_rating}')

In [ ]:
def match_cases(array1, wanted_array1, array2, wanted_array2):
    """Notice de diference: the original data has to be of the samen lenght,
    BUT the match cases can differ. This means that I can have 3 match cases
    on one side and 50 in the other, and it won't make any trouble"""
    if len(array1)==len(array2):
        match_cases = []

        #Creating Support dictionaries
        wanted_cases_1 = {}
        for data in range(len(wanted_array1)):
            wanted_cases_1[wanted_array1[data]] = 1
        
        wanted_cases_2 = {}
        for data in range(len(wanted_array2)):
            wanted_cases_2[wanted_array2[data]] = 1
        
        for ub in range(len(array1)):
            val1 = array1[ub]
            val2 = array2[ub]
            if val1 in wanted_cases_1 and val2 in wanted_array2:
                match_cases.append(ub)
            
        return match_cases

    else: return 'Arrays must have the samen lenght'

Con la función diseñada, procedemos a insertar los arreglos originales, acompañados de los sets en los que se encuentran los valores permitidos. Cabe resaltar que esta función solo encuentra la ubicación de los valores donde nuestros universos y condiciones se encuentran, más no otorga el libro resultante, ya que no se introduce este dato en la función.

In [ ]:
match_cases = match_cases(rating, topc_rating, reviews, topc_reviews)

Estos casos, denominados "mejor evaluados" (bv), se almacenarán un DataFrame distinto, para analizar de forma más fácil su output.

In [ ]:
bv_rate = []
for ub in match_cases:
    bv_rate.append(rating[ub])

bv_reviews = []
for ub in match_cases:
    bv_reviews.append(reviews[ub])    
    
bv_titles = []
for ub in match_cases:
    bv_titles.append(titles[ub])
    
bv_books_dict = {'rating': bv_rate,
             'reviews': bv_reviews,
             'titles': bv_titles}

bv_books_df = pd.DataFrame(bv_books_dict, columns=['rating','reviews','titles'])
bv_books_df

Ordenando los valores según la columna de reviews, podremos obtener el libro con mayor número de reviews y que tenga una calificación mayor a 4.5. Considerándolo por ende, uno de los mejores libros calificados en este Data Set

In [ ]:
bv_books_df.sort_values(by = 'reviews', ascending = False)

<h3>Caso contrario. Solo considerar Reviews</h3>

Los siguientes chuncks fueron diseñados para obtener el "top" deseado de una serie de elementos numéricos, permitiendo la observación de una tabla dónde solo se consideran los 7 libros con más reviews. Este número fue escogido por la cantidad de coincidencias obtenidas para el DataFrame anterior.

In [ ]:
def topx(array, top_size):
    """Considerations.
    1) Array is ORDERED from minor to major.
    2) You can insert Arrays with duplicated values.
    3) You won't insert Sets."""

    sorted_and_reduced_array = sorted_set(array.copy())
    position = len(sorted_and_reduced_array) - 1
    
    topx = []
    value = sorted_and_reduced_array[position]
    topx.append(value)

    position -= 1
    max_value = value
    value = sorted_and_reduced_array[position]

    top_values = 1 

    while top_values < top_size and position >= 0 :
        if value != max_value:
            topx.append(sorted_and_reduced_array[position])
            max_value = value
            top_values += 1
        position -= 1
        value = sorted_and_reduced_array[position]

    return topx

In [ ]:
top7_reviews = topx(reviews, 7)

In [ ]:
def match_top_cases(array1, wanted_array1):
        match_cases = []

        #Creating Support dictionaries
        wanted_cases_1 = {}
        for data in range(len(wanted_array1)):
            wanted_cases_1[wanted_array1[data]] = 1
        
        for ub in range(len(array1)):
            val1 = array1[ub]
            if val1 in wanted_cases_1:
                match_cases.append(ub)
            
        return match_cases

In [ ]:
ub_top7r = match_top_cases(reviews, top7_reviews)

In [ ]:
def array_matches(match_array, array1, array2, array3):
    
    match_array1 = []
    for ub in match_array:
        match_array1.append(array1[ub])

    match_array2 = []
    for ub in match_array:
        match_array2.append(array2[ub])    

    match_array3 = []
    for ub in match_array:
        match_array3.append(array3[ub])
    
    return match_array1,match_array2,match_array3

In [ ]:
t7rev_rating, t7rev_reviews, t7rev_titles = array_matches(ub_top7r, rating, reviews, titles)
    
top7_books_dict = {'rating': t7rev_rating,
             'reviews': t7rev_reviews,
             'titles': t7rev_titles}

top7_books_df = pd.DataFrame(top7_books_dict, columns=['rating','reviews','titles'])
top7_books_df.sort_values(by = 'reviews', ascending = False)

Con esta tabla de resultado, es posible apreciar que el hecho, de que un libro posea un alto número de receñas, no es garantía de una buena calificación, o que sea altamente recomendado.

<h2>Comportamiento rating vs reviews</h2>

Debido a los resultados anteriores, se decide implementar un gráfico de puntos "rating vs reviews". Este gráfico podrá servir para explicar mejor lo que sucede si solo se consideran el número de receñas.

In [ ]:
books_df

In [ ]:
fig = books_df.plot(kind="scatter", x='rating', y='reviews', c='green')

x = books_df['rating']
plt.axvline(np.mean(x)-np.std(x), c = 'k', linestyle = ':', label = '-1 desv. std.')
plt.axvline(np.mean(x), c = 'r', linestyle = '-', label = 'Promedio')
plt.axvline(np.mean(x)+np.std(x), c = 'k', linestyle = ':', label = '+1 desv. std.')

y = books_df['reviews']
plt.axhline(np.mean(y)-np.std(y), c = 'c', linestyle = ':', label = 'Y: -1 desv. std.')
plt.axhline(np.mean(y), c = 'b', linestyle = '-', label = 'Promedio y')
plt.axhline(np.mean(y)+np.std(y), c = 'c', linestyle = ':', label = 'Y: +1 desv. std.')

fig.legend()

Este gráfico presenta una condensación de los puntos entre una calificación de 3.5 y 4.5 y menores a 20,000 reviews. Para el caso de libros mayores a 4.5 podemos observar que no existe un repunte ni libros que traspasen dicho rango, pero sí que los puntos con mayor número de reviews se encuentran igual en dicho rango.

<h1>Conclusiones</h1>

<ol>
<li>Un libro tenga una calificación de 5 estrellas no es garantía de que a mucha gente le haya parecido un libro excelente</li>
<li>Que un libro haya sido leído por mucha gente, no garantiza que a todos les haya parecido un libro excelente</li>
<li>Siempre hay que considerar todas las variables disponibles y relevantes de un Data Set</li>
<li>Conforme más condiciones agreguemos a nuestros análisis, nuestras deducciones pueden ser más precisas y objetivas</li>
</ol>